In [ ]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

In [ ]:
def answer_one():
    import pandas as pd
    import numpy as np
    spam_count = np.bincount(spam_data['target'])[1]
    total_count = np.bincount(spam_data['target'])[0]+np.bincount(spam_data['target'])[1]
    ratio = spam_count / total_count
    return ratio * 100

In [ ]:
answer_one()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    from sklearn.feature_extraction.text import CountVectorizer
    vect = CountVectorizer()
    X_train_vect = vect.fit_transform(X_train)
    feature_names = np.array(vect.get_feature_names())
    length_tokens = []
    for i in range(len(feature_names)):
        length_tokens.append(len(feature_names[i]))
    length_tokens.sort()
    length_longest_token = length_tokens[-1]
    for i in range(len(feature_names)):
        if len(feature_names[i]) == length_longest_token:
            longest_word = feature_names[i]
    
    return longest_word

In [ ]:
answer_two()

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.metrics import roc_auc_score

    vect = CountVectorizer()
    X_train_vect = vect.fit_transform(X_train)
    model = MultinomialNB(alpha = 0.1)
    model.fit(X_train_vect, y_train)
    predictions = model.predict(vect.transform(X_test))
    score = roc_auc_score(y_test, predictions)
    
    return score

In [ ]:
answer_three()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    from sklearn.feature_extraction.text import TfidfVectorizer

    vect_tfidf = TfidfVectorizer()
    X_train_tfidf = vect_tfidf.fit_transform(X_train)
    feature_names_tfidf = np.array(vect_tfidf.get_feature_names())
    sorted_tfidf_index = X_train_tfidf.max(0).toarray()[0].argsort()
    smallest_tfidf = feature_names_tfidf[sorted_tfidf_index[:20]]
    largest_tfidf = feature_names_tfidf[sorted_tfidf_index[:-21:-1]]
    new_list = X_train_tfidf.max(0).toarray()[0]
    smallest_val_tfidf = np.sort(new_list)[:20]
    largest_val_tfidf = np.sort(new_list)[:-21:-1]
    series1 = pd.Series(smallest_val_tfidf, index = smallest_tfidf)
    series2 = pd.Series(largest_val_tfidf, index = largest_tfidf)
    return (series1.sort_index(), series2.sort_index())

In [ ]:
answer_four()

In [ ]:
def answer_five():
    
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.metrics import roc_auc_score

    vect = TfidfVectorizer(min_df = 3).fit(X_train)
    X_train_vect = vect.transform(X_train)

    model  = MultinomialNB(alpha = 0.1)
    model.fit(X_train_vect, y_train)

    predictions = model.predict(vect.transform(X_test))
    score_new = roc_auc_score(y_test, predictions)
    return score_new

In [ ]:
answer_five()

In [ ]:
def answer_six():
    doc_len = []
    count = 0
    for i in range(len(spam_data[spam_data['target'] == 1])):
        doc_len.append(len(spam_data[spam_data['target'] == 1]['text'].iloc[i]))
        count = count + doc_len[i]
    avrg_spam = count/len(spam_data[spam_data['target']==1])
    doc_len_nospam = []
    count_nospam = 0
    for i in range(len(spam_data[spam_data['target'] == 0])):
        doc_len_nospam.append(len(spam_data[spam_data['target'] == 0]['text'].iloc[i]))
        count_nospam = count_nospam + doc_len_nospam[i]
    avrg_no_spam = count_nospam/len(spam_data[spam_data['target']==0])  
    
    return (avrg_no_spam, avrg_spam)

In [ ]:
answer_six()

In [ ]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [ ]:
from sklearn.svm import SVC

def answer_seven():
    from sklearn.svm import SVC
    from sklearn.metrics import roc_auc_score
    spam_data['new_feature'] = 0
    for i in range(len(spam_data)):        
        spam_data['new_feature'][i] = len(spam_data['text'][i])
    X_train, X_test, y_train, y_test = train_test_split(spam_data[['text', 'new_feature']],
                                                        spam_data['target'],
                                                        random_state = 0)
    vect = TfidfVectorizer(min_df = 5)
    vect.fit(X_train['text'])
    X_vect_train = vect.transform(X_train['text'])
    add_feature(X_vect_train, X_train['new_feature'])
    model = SVC(C = 10000).fit(add_feature(X_vect_train, X_train['new_feature']), y_train)
    predictions = model.predict(add_feature(vect.transform(X_test['text']), X_test['new_feature']))
    score_auc = roc_auc_score(y_test, predictions)
    return score_auc

In [ ]:
answer_seven()

In [ ]:
def answer_eight():
    import re
    spam_data['nr_digit'] = 0
    spam_data['nr_digit'] = spam_data['text'].str.count(r'\d').tolist()
    count_spam = 0
    count_nospam = 0
    for i in range(len(spam_data)):
        if spam_data['target'][i] == 1:
            count_spam = count_spam + spam_data['nr_digit'][i]
        else:
            count_nospam = count_nospam + spam_data['nr_digit'][i]
    total_spam = len(spam_data[spam_data['target'] == 1])
    total_nospam = len(spam_data[spam_data['target'] == 0])
    avrg_dig_nospam = count_nospam / total_nospam
    avrg_dig_spam = count_spam / total_spam
    return (avrg_dig_nospam, avrg_dig_spam)

In [ ]:
answer_eight()

In [ ]:
def answer_nine():
    from sklearn.svm import SVC
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_auc_score
    import re
    spam_data['nr_digit'] = 0
    spam_data['nr_digit'] = spam_data['text'].str.count(r'\d').tolist()
    count_spam = 0
    count_nospam = 0
    for i in range(len(spam_data)):
        if spam_data['target'][i] == 1:
            count_spam = count_spam + spam_data['nr_digit'][i]
        else:
            count_nospam = count_nospam + spam_data['nr_digit'][i]
    spam_data['new_feature'] = 0
    for i in range(len(spam_data)):        
        spam_data['new_feature'][i] = len(spam_data['text'][i])
    
    avrg_digit_number = (count_spam + count_nospam) / (len(spam_data))
    spam_data['new_feature2'] = avrg_digit_number
    X_train, X_test, y_train, y_test = train_test_split(spam_data[['text', 'new_feature', 'new_feature2']],
                                                        spam_data['target'],
                                                        random_state = 0)
    vect = TfidfVectorizer(min_df = 5, ngram_range = (1,3))
    vect.fit(X_train['text'])
    X_vect_train = vect.transform(X_train['text'])
    new_X_train = add_feature(X_vect_train, X_train['new_feature'])
    add_feature(new_X_train, X_train['new_feature2'])
    new_X_test = add_feature(vect.transform(X_test['text']), X_test['new_feature'])
    model = LogisticRegression(C = 100).fit(add_feature(new_X_train, X_train['new_feature2']), y_train)
    predictions = model.predict(add_feature(new_X_test, X_test['new_feature2']))
    score_auc = roc_auc_score(y_test, predictions)
    return score_auc

In [ ]:
answer_nine()

In [ ]:
def answer_ten():
    spam_data['nr_noword'] = 0
    spam_data['nr_noword'] = spam_data['text'].str.count(r'\W').tolist()
    count_spam_noword = 0
    count_nospam_noword = 0
    for i in range(len(spam_data)):
        if spam_data['target'][i] == 1:
            count_spam_noword = count_spam_noword + spam_data['nr_noword'][i]
        else:
            count_nospam_noword = count_nospam_noword + spam_data['nr_noword'][i]
    total_spam_noword = len(spam_data[spam_data['target'] == 1])
    total_nospam_noword = len(spam_data[spam_data['target'] == 0])
    avrg_nw_nospam = count_nospam_noword / total_nospam_noword
    avrg_nw_spam = count_spam_noword / total_spam_noword
    return (avrg_nw_nospam, avrg_nw_spam)

In [ ]:
answer_ten()

In [ ]:
def answer_eleven():
    
    
    return #Your answer here

In [ ]:
answer_eleven()